In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
from scipy import stats

# calculate the distance between p1 and p2 under boundary conditions
def cal_bc_distance(p1,p2):
    t1 = 0
    t2 = 0
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    if dx > L/2:
        t1 = -L
    elif dx < -L/2:
        t1 = L
    if dy > L/2:
        t2 = -L
    elif dy < -L/2:
        t2 = L    
    bc_dist = np.sqrt((dx + t1)**2+(dy + t2)**2)
    return bc_dist

# return the distance matrix(N*N) calculated from birds positions
def cal_pdist(birds_pos,N):
    Dist = np.zeros((N,N))
    for i in range(1,N):
        for j in range(i):
            Dist[i,j] = Dist[j,i] = cal_bc_distance(birds_pos[i,:],birds_pos[j,:])
    return Dist

# initialize the state of birds
def initialize(L,N):
    state = np.zeros((N,3))
    state[:,:2] = np.random.uniform(0,L,(N,2))# positions x,y
    state[:,2] = np.random.uniform(0,2*np.pi,N) # angles theta
    return state

# update the state of birds in each time step
def update(state, L, N, v0, R, eta): 
    # Initialise heading with noise
    heading = eta
    
    # Use adjacency matrix to determine neighbours
    A = cal_pdist(state[:,:2],N)
    for i in range(N):
        adj = np.where(A[i,:] < R)[0] # indices of adjacent particles
        theta = state[adj,2] # angles of all adjacent particles
        
        # Sum sin and cos of angles
        sum_sin = np.sum(np.sin(theta))
        sum_cos = np.sum(np.cos(theta))
        
        # Compute heading for this particle
        heading[i] += np.arctan2(sum_sin, sum_cos)
        
    # Update state with new headings
    state[:,2] = heading 
    # Update positions(mod L)
    state[:,0] = (state[:,0] + v0*np.cos(state[:,2]))%L
    state[:,1] = (state[:,1] + v0*np.sin(state[:,2]))%L
    return state

# calculate the Vicsek order parameter
def order_para(state,N):
    x = np.sum(np.cos(state[:,2]))
    y = np.sum(np.sin(state[:,2]))
    order_para = np.sqrt(x**2+y**2)/N
    return order_para




In [ ]:
N = 200  # number of individual birds
L = 15   # size of the container(L*L)
v0 = 0.5  # the constant velocity of the birds per time step
R = 1 # radius within which to search for neighbours
sigma = np.pi/8  # variance of noise
T = 10000  # total time steps
Noise_generator = stats.norm(0,np.sqrt(sigma))
opara = np.zeros(T) # the Vicsek order parameter

state = initialize(L,N)
plt.figure(figsize=(6,24))
for t in range(T):
    eta = Noise_generator.rvs(N) # noise term
    state = update(state, L, N, v0, R, eta)
    opara[t] = order_para(state,N)
    
    #test: plot the graph at time step：999 ，3999，6999，9999
    i = 1
    if (t-999)%3000 == 0:
        plt.subplot(4,1,i)
        i += 1
        plt.quiver(state[:,0],state[:,1],np.cos(state[:,2]),np.sin(state[:,2]),np.arange(N)) 
        plt.xlim(0,15)
        plt.ylim(0,15)
        plt.show()


plt.figure(figsize=(3,5))
plt.plot(np.arange(T),opara,'b')    
plt.show()

In [ ]:
N = 200  # number of individual birds
L = 15   # size of the container(L*L)
v0 = 0.5  # the constant velocity of the birds per time step
R = 1 # radius within which to search for neighbours
sigma = 1  # variance of noise
T = 10000  # total time steps
Noise_generator = stats.norm(0,np.sqrt(sigma))
opara = np.zeros(T) # the Vicsek order parameter

state = initialize(L,N)
for t in range(T):
    eta = Noise_generator.rvs(N) # noise term
    state = update(state, L, N, v0, R, eta)
    #opara[t] = order_para(state,N)
    fig = plt.figure()
    ax = plt.subplot(111)
    ax.quiver(state[:,0],state[:,1],np.cos(state[:,2]),np.sin(state[:,2]),np.arange(N)) 
    ax.set_xlim(0,15)
    ax.set_ylim(0,15)
    fig.savefig('pics/{}.png'.format(t))


In [ ]:
# make a video
import cv2
import numpy as np

img_array = []
#for filename in glob.glob('pics/*.png'):
for t in range(T):
    #img = cv2.imread(filename)
    img = cv2.imread('pics/{}.png'.format(t))
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

out = cv2.VideoWriter('project.mp4',cv2.VideoWriter_fourcc(*'FMP4'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()